In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import numpy as np
import pandas as pd
import time

: 

In [2]:
start=time.time()
targetSites = ['https://www.munpia.com/page/best/section/plsa.eachtoday', 'https://www.munpia.com/page/best/section/plsa.weekly',
               'https://www.munpia.com/page/best/section/plsa.monthly', 'https://www.munpia.com/page/best/section/plsa.newbie']

for targetSite in targetSites:
    driver = webdriver.Chrome() 
    driver.get(targetSite)
    time.sleep(1)
    
    for i in range(10): # 가장 마지막까지 스크롤 200위까지만 나옴
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
    
    # 원래는 순위대로 하나씩 클릭하고 크롤링 후 back()하려 했으나 다시 아래로 스크롤 해야 해서 
    # 스크롤 한 다음 하이퍼링크 주소를 따와서 리스트에 저장한 후 하나씩 열고 크로링 하기로 함.
    M_Links = []
    M_Link_List = driver.find_elements(By.CLASS_NAME, 'best-item')
    for M_Link in M_Link_List:
        M_Links.append(M_Link.get_attribute('href'))
    M_Links.remove(M_Links[3]) 
    # 하이퍼 링크 저장 함=========================================================================
    
    # 리스트 초기화
    Title = [] # 타이틀
    Genre = [] # 장르
    Author = [] # 작가
    Start = [] # 연재 시작일
    Last = [] # 마지막 연재일
    Count = [] # 연재수
    View = [] # 조회수
    Good = [] # 추천수
    Typing = [] # 글자수
    Favorite = [] # 선호작 수
    
    for i in range(len(M_Links)):
        driver.get(M_Links[i]) # 0부터 하나씩 저장된 하이퍼링크로 이동

        Title.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/h2/div/a').text) # 제목 추가
        Genre.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/p[1]/strong').text) # 장르 추가
        
        try:
            Author.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[1]/dd/a/strong').text) # 작가 추가
        except:
            Author.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[1]/dd').text) # 작가 추가
            
        try:
            Favorite.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/div[2]/div[2]/a[1]/span/b').text) # 선작수 추가
        except:
            Favorite.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/div/div[2]/a[1]/span/b').text) # 선작수 추가    
            
        Start.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[2]/dd[1]').text) # 연재 시작일 추가
        Last.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[2]/dd[2]').text) # 마지막 연재일 추가

        Count.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[3]/dd[1]').text) # 연재수 추가
        View.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[3]/dd[2]').text) # 조회수 추가
        Good.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[3]/dd[3]').text) # 추천수 추가
        Typing.append(driver.find_element(By.XPATH, '//*[@id="board"]/div[1]/div[3]/dl[3]/dd[4]').text) # 글자수 추가
        print(f'{i}. 제목: {Title[i]}, 장르: {Genre[i]}, 작가: {Author[i]}, 선작수: {Favorite[i]}, 시작일: {Start[i]}, 최근화: {Last[i]}, 회수: {Count[i]}, 조회수: {View[i]}, 추천수: {Good[i]}, 글자수: {Typing[i]}')
        
    print(len(Title), len(Genre), len(Author), len(Favorite), len(Start), len(Last), len(Count), len(View), len(Good), len(Typing))
    
    df= pd.DataFrame({"제목":Title, "장르":Genre, "작가":Author, "연재 시작일":Start, "마지막 연재일":Last, "회수":Count, "조회수":View, "추천수":Good, "글자수":Typing, "선작수":Favorite})
    df.index = df.index+1 
    
    # 데이터프레임으로 저장 및 인덱스 1부터 시작
    # 데이터프레임을 data 폴더에 저장함 근데 4개(today, week, month, new) 따로 저장 하도록 함
    if targetSite == 'https://www.munpia.com/page/best/section/plsa.eachtoday':
        df.to_csv('./data/M_Today_Best200.txt')
    elif targetSite == 'https://www.munpia.com/page/best/section/plsa.weekly':
        df.to_csv('./data/M_Week_Best200.txt')
    elif targetSite == 'https://www.munpia.com/page/best/section/plsa.monthly':
        df.to_csv('./data/M_Month_Best200.txt')
    else:
        df.to_csv('./data/M_New_Best200.txt') 
end=time.time()     
print(end-start)

0. 제목: 선독점 종말의 뱀이 되었다, 장르: 판타지, 퓨전, 작가: 비도™, 선작수: 32,846, 시작일: 2024.05.08 10:05, 최근화: 2024.06.30 18:00, 회수: 59 회, 조회수: 3,062,265, 추천수: 118,115, 글자수: 377,646
1. 제목: 선독점 나의 유럽 원정기, 장르: 현대판타지, 작가: 동쪽사람, 선작수: 14,560, 시작일: 2024.05.08 12:39, 최근화: 2024.06.30 11:00, 회수: 59 회, 조회수: 1,511,194, 추천수: 52,712, 글자수: 357,220
2. 제목: 선독점 아카식 경제학, 장르: 현대판타지, 퓨전, 작가: 드림보트, 선작수: 16,042, 시작일: 2024.05.08 10:30, 최근화: 2024.06.29 22:00, 회수: 56 회, 조회수: 1,882,622, 추천수: 52,705, 글자수: 365,773
3. 제목: 선독점 방구석 저격수., 장르: 현대판타지, 퓨전, 작가: 캡슐호텔, 선작수: 23,512, 시작일: 2024.04.19 23:20, 최근화: 2024.06.29 22:01, 회수: 77 회, 조회수: 2,719,176, 추천수: 65,767, 글자수: 571,025
4. 제목: 선독점 무신연의 : 여포가 효도를 잘함, 장르: 퓨전, 대체역사, 작가: 별꽃라떼, 선작수: 26,416, 시작일: 2024.05.08 11:55, 최근화: 2024.06.30 18:00, 회수: 79 회, 조회수: 3,093,960, 추천수: 126,285, 글자수: 714,014
5. 제목: 선독점 범상한 변호사의 아공간, 장르: 현대판타지, 작가: 서칸더브이, 선작수: 19,684, 시작일: 2024.01.04 14:53, 최근화: 2024.06.30 09:00, 회수: 185 회, 조회수: 3,478,014, 추천수: 121,423, 글자수: 1,042,318
6. 제목: 선독점 이상적 연출생활백서, 장르: 현대판타지, 드라마, 작가: 자경(蔗境